# Importing Dependencies

In [187]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import docx2txt
import neattext as nt
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.stem import PorterStemmer
ps =PorterStemmer()
from nltk.stem import 	WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
cv=CountVectorizer()
from sklearn.metrics.pairwise import cosine_similarity

In [92]:
files=os.listdir('./Anonymous_CVs/')

In [94]:
documents=dict()

In [95]:
for file in files[0:5]:
    document= docx2txt.process('./Anonymous_CVs/'+file)
    documents[file]=document

In [99]:
documents_df=pd.DataFrame(documents.items(),columns=['Document Name','Documents'])

In [100]:
documents_df

,Document Name,Documents
0,CV1.docx,Donald Petrovich\n\nEmail: DonaldPetrovich@gma...
1,CV2.docx,Helen Grant\n\n(922) 679-9797\nHelenGrant@gmai...
2,CV3.docx,Clarence Price\n\n(786) 324-2395 ClarencePri...
3,CV4.docx,Jennifer Gillman\n\nJenniferGillman@gmail.com\...
4,CV5.docx,Bonnie Pelt\n \t\t Email/Skype: Bonnie...


In [116]:
def docs_cleaning(text):
    text=nt.remove_emails(text)
    text=nt.remove_numbers(text)
    text=nt.remove_stopwords(text)
    text=nt.remove_special_characters(text)
    text=nt.remove_emojis(text)
    text=nt.remove_phone_numbers(text)
    text=nt.remove_multiple_spaces(text)
    return text.lower()

In [117]:
documents_df['Cleand_Documents']=documents_df['Documents'].apply(lambda x:docs_cleaning(x))

In [128]:
import nltk
from nltk.stem import 	WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [129]:
lemmarized_words=[]

In [130]:
def lemmatization(text):
    lemmarized_words=[]
    tokenization = nltk.word_tokenize(text)
    for word in tokenization:
        word=wordnet_lemmatizer.lemmatize(word)
        lemmarized_words.append(word)
    return ' '.join(lemmarized_words)
    

In [132]:
documents_df['Lemmatized_Documents']=documents_df['Cleand_Documents'].apply(lambda x:lemmatization(x))

In [153]:
documents_df['Lemmatized_Documents']

0    donald petrovich email phone current location ...
1    helen grant objective year strong software exp...
2    clarence price project manager experienced pro...
3    jennifer gillman certified scrum master busine...
4    bonnie pelt emailskype mob professional summar...
Name: Lemmatized_Documents, dtype: object

In [164]:
word_dict=dict()

In [168]:
def word_dictionary(text):
    for word in text.split():
        if word in word_dict.keys():
            word_dict[word]+=1
        else:
            word_dict[word]=1
    return word_dict

In [169]:
documents_df['Word_Frequencies']=documents_df['Lemmatized_Documents'].apply(lambda x:word_dictionary(x))

In [196]:
def pre_processing(text):
    text=nt.remove_emails(text)
    text=nt.remove_numbers(text)
    text=nt.remove_stopwords(text)
    text=nt.remove_special_characters(text)
    text=nt.remove_emojis(text)
    text=nt.remove_phone_numbers(text)
    text=nt.remove_multiple_spaces(text)
    text=text.lower()
    tokenization = nltk.word_tokenize(text)
    lemmarized_words=[]
    for word in tokenization:
        word=wordnet_lemmatizer.lemmatize(word)
        lemmarized_words.append(word)
    return ' '.join(lemmarized_words)

In [197]:
document= docx2txt.process('./Anonymous_CVs/CV1.docx')

In [198]:
document=pre_processing(document)

In [175]:
def word_dictionary(text):
    for word in text.split():
        if word in word_dict.keys():
            word_dict[word]+=1
        else:
            word_dict[word]=1
    return word_dict

In [176]:
document_dict=word_dictionary(document)

In [212]:
def get_similar_docs(documents,searched_document):
    content=[list(documents.values()),list(searched_document.values())]
    #matrix=cv.fit_transform(content)
    similarity_matrix=cosine_similarity(content)[0][1]
    similarity=round((similarity_matrix*100),3)
    return similarity

In [213]:
documents_df['Sim']=documents_df['Word_Frequencies'].apply(lambda x:get_similar_docs(x,document_dict))

In [202]:
documents_df['Sim']

0    100.000
1     82.774
2     10.857
3     21.013
4     31.377
Name: Sim, dtype: float64